# Genie Deep Dive: Making Your Semantic Model Conversational

Now that you've created a semantic model with metric views, it's time to make it conversational with Genie. Leadership wants you to enable natural language queries so anyone can ask questions about the IoT data without writing SQL.

## What You'll Learn

✅ Create a Genie Space for your semantic model  
✅ Add instructions to guide query generation  
✅ Define filters for common questions  
✅ Test with a benchmark query  

---

## The Scenario

Your semantic model (`iot_sensor_metrics`) is ready, but the operations team doesn't know SQL. They need to ask questions like:
- "What's the average temperature in the West region?"
- "Show me devices with high anomaly rates"
- "Which factory has the most critical readings?"

Genie will translate these natural language questions into queries against your semantic model.

---

## What is Genie?

**Genie** is Databricks' AI-powered natural language interface for querying data. It uses your semantic model and additional context to generate accurate SQL queries.

**Key Components for This Exercise:**
- **Genie Space**: Container pointing to your metric view
- **Instructions**: Business context and terminology
- **Filters**: Pre-defined query shortcuts
- **Benchmark**: Test question to verify accuracy

---

**References:**
- [Genie Setup](https://docs.databricks.com/aws/en/genie/set-up)
- [Best Practices](https://docs.databricks.com/aws/en/genie/best-practices)

## Step 1: Create Your Genie Space  

### Setup in the UI

**Navigate to Genie:**
1. Click **Genie** in the sidebar
2. Click **Create Space**

**Add Your Semantic Model:**
- Select `default.db_crash_course.iot_sensor_metrics` (the metric view you created in notebook 1)
- Optionally add `dim_factories` if you need factory details beyond what's in the metric view

**Why use the metric view?**
- ✅ Pre-defined metrics are already understood by Genie
- ✅ Consistent business logic
- ✅ Faster queries (uses aggregated data)
- ✅ Better accuracy (clear metric definitions)

---

**⚠️ Keep it simple**: Just the metric view is enough to get started. You can add more tables later if needed.

## Step 2: Add Instructions

### What are Instructions?

**Instructions** teach Genie about your business context and terminology. They're shown to Genie when generating queries.

### Add Business Context

In your Genie Space, click **Instructions** and add this context:

```
## IoT Sensor Data Context

You are helping analyze data from aircraft sensors deployed across factories.

### Key Metrics (from iot_sensor_metrics)
- average_temperature: Average temperature in Fahrenheit across sensors
- total_sensor_readings: Total count of all sensor readings
- active_devices: Number of distinct devices reporting data

### Dimensions
- factory_name: Name of the factory location
- region: Geographic region (West, East, Central, International)
- model_category: Aircraft model category
- reading_date: Date of sensor reading

### Business Rules
- Critical threshold: Temperature > 85°F
- Normal operating range: 60-75°F
- Operating hours: 6 AM - 10 PM daily

### Join Keys
- factory_id: Connects to dim_factories table for additional factory details
```

**Why this helps:**
- ✅ Genie understands your metric definitions
- ✅ Knows what thresholds matter
- ✅ Can join to related tables when needed
- ✅ Generates more accurate queries

---

**💡 Tip**: Keep instructions concise. Too much information can confuse Genie.

## Step 3: Define Filter Shortcuts 

### What are Filters?

**Filters** are pre-defined query shortcuts that help Genie understand common data slices.

### Add Common Filters

In your Genie Space, add these filters:

**Region Filter:**
```
Name: West Region
Description: Filter for Western region factories
SQL: region = 'West'
```

**Time Filter:**
```
Name: Last 7 Days
Description: Data from the past week
SQL: reading_date >= CURRENT_DATE - 7
```

**Critical Readings:**
```
Name: Critical Temperatures
Description: Readings above critical threshold
SQL: average_temperature > 85
```

### How Users Will Use Filters

With these filters defined, users can ask:
- "Show me average temperature for **West Region**"
- "What's the device count in the **Last 7 Days**?"
- "Count **Critical Temperatures** by factory"

Genie will automatically apply the correct SQL filter.

---

**💡 Pro Tip**: Start with 2-3 essential filters. Add more based on actual user questions.

## Step 4: Test with a Benchmark

### What is a Benchmark?

A **benchmark** is a test question used to verify Genie is generating accurate queries. It helps you measure and improve quality.

### Create Your First Benchmark

In your Genie Space, navigate to **Benchmarks** and add this test question:

```
Question: What's the average temperature in the West region over the last 7 days?

Expected Answer Type: Single number (should be around 60-75°F if data is normal)

Why this tests multiple features:
- Uses the metric view (average_temperature)
- Applies a region filter (West)
- Includes a time filter (last 7 days)
- Tests that Genie understands your dimensions
```

### Run the Benchmark

1. Click **Run Benchmark**
2. Review the generated SQL
3. Check if the answer is reasonable

**What Good SQL Should Look Like:**
```sql
SELECT average_temperature
FROM default.db_crash_course.iot_sensor_metrics
WHERE region = 'West' 
  AND reading_date >= CURRENT_DATE - 7
```

### Improve if Needed

If the benchmark fails:
- ✅ Check your instructions are clear
- ✅ Verify filter definitions match your metric view columns
- ✅ Ensure the metric view is available in your Genie Space
- ✅ Add more specific examples to instructions

---

**✅ Success Criteria**: Genie generates the correct SQL and returns a reasonable temperature value.


In [ ]:
-- Example 1: Factory Performance
-- Get average temperature and device count by factory
SELECT 
  factory_name,
  region,
  AVG(average_temperature) as avg_temp,
  SUM(active_devices) as device_count
FROM dwx_airops_insights_platform_dev_workspace.db_crash_course.iot_sensor_metrics
WHERE reading_date >= CURRENT_DATE - 30
GROUP BY factory_name, region
ORDER BY avg_temp DESC


In [ ]:
-- Example 2: Critical Temperature Trend
-- Show factories with temperatures above threshold over time
SELECT 
  reading_date,
  factory_name,
  average_temperature
FROM dwx_airops_insights_platform_dev_workspace.db_crash_course.iot_sensor_metrics
WHERE average_temperature > 85
  AND reading_date >= CURRENT_DATE - 7
ORDER BY reading_date, factory_name


In [ ]:
-- Example 3: Regional Comparison
-- Compare metrics across regions
SELECT 
  region,
  COUNT(DISTINCT factory_name) as factory_count,
  AVG(average_temperature) as avg_temp,
  SUM(total_sensor_readings) as total_readings
FROM dwx_airops_insights_platform_dev_workspace.db_crash_course.iot_sensor_metrics
WHERE reading_date >= CURRENT_DATE - 30
GROUP BY region
ORDER BY avg_temp DESC


**Why These Examples Help:**
- Genie learns your preferred query patterns
- Complex joins are handled consistently
- Business logic is embedded in examples
- Users get better results without knowing SQL

**Reference:** [SQL Query Examples Documentation](https://docs.databricks.com/aws/en/genie/trusted-assets)

---


## Summary and Next Steps

### What You Accomplished

In 30 minutes, you:

✅ **Created a Genie Space** pointing to your semantic model  
✅ **Added instructions** with business context and terminology  
✅ **Defined 3 filters** for common query patterns  
✅ **Tested with a benchmark** to verify accuracy  

### Your Genie Space is Now Ready!

Users can now ask questions like:
- "What's the average temperature by factory?"
- "Show me active devices in the West region"
- "What are the critical temperatures in the last week?"

No SQL required! Genie translates these to queries against your `iot_sensor_metrics` semantic model.

---

## Try This Out (Optional Extensions)

Want to go further? Try these enhancements:

### 1. Add More Filters
Create filters for:
- Different regions (East, Central, International)
- Time ranges (last 30 days, last month)
- Model categories

### 2. Create Additional Benchmarks
Test more complex scenarios:
- "Which factory has the highest average temperature?"
- "Show me device count trends over time"
- "Compare West vs East regions"

### 3. Add Sample Q&A
In the Instructions, add example questions and expected results to improve accuracy.

### 4. Add SQL Query Examples as Trusted Assets

SQL Query Examples help Genie understand how to write queries for your specific data model. Add them as trusted assets:

**What are SQL Query Examples?**
- Pre-written SQL queries that show Genie how to query your data
- Help with complex joins, aggregations, or business logic
- Act as templates that Genie can adapt to similar questions

**How to Add:**
1. In your Genie Space, navigate to **Trusted Assets**
2. Click **Add SQL Query Example**
3. Add these examples:

**Example 1: Factory Performance**
```sql
-- Get average temperature and device count by factory
SELECT 
  factory_name,
  region,
  AVG(average_temperature) as avg_temp,
  SUM(active_devices) as device_count
FROM default.db_crash_course.iot_sensor_metrics
WHERE reading_date >= CURRENT_DATE - 30
GROUP BY factory_name, region
ORDER BY avg_temp DESC
```

**Example 2: Critical Temperature Trend**
```sql
-- Show factories with temperatures above threshold over time
SELECT 
  reading_date,
  factory_name,
  average_temperature
FROM default.db_crash_course.iot_sensor_metrics
WHERE average_temperature > 85
  AND reading_date >= CURRENT_DATE - 7
ORDER BY reading_date, factory_name
```

**Example 3: Regional Comparison**
```sql
-- Compare metrics across regions
SELECT 
  region,
  COUNT(DISTINCT factory_name) as factory_count,
  AVG(average_temperature) as avg_temp,
  SUM(total_sensor_readings) as total_readings
FROM default.db_crash_course.iot_sensor_metrics
WHERE reading_date >= CURRENT_DATE - 30
GROUP BY region
ORDER BY avg_temp DESC
```

**Why This Helps:**
- Genie learns your preferred query patterns
- Complex joins are handled consistently
- Business logic is embedded in examples
- Users get better results without knowing SQL

**Reference:** [SQL Query Examples Documentation](https://docs.databricks.com/aws/en/genie/trusted-assets)

### 5. Join to More Tables
If you added `dim_factories`, test questions that need factory details:
- "What's the temperature at factories in California?"
- "Show me metrics for factories with more than 100 devices"

---

## Key Takeaways

**Semantic Model + Genie = Self-Service Analytics**
1. **Semantic model** (notebook 1) defines consistent metrics
2. **Genie** (this notebook) makes them conversational
3. **Result**: Non-technical users can explore data

**Start Simple, Iterate**
- Begin with essential context
- Add filters as users need them
- Use benchmarks to measure quality
- Improve based on actual usage


---

**Additional Resources:**
- [Genie Documentation](https://docs.databricks.com/aws/en/genie/)
- [Best Practices](https://docs.databricks.com/aws/en/genie/best-practices)
- [Knowledge Store Guide](https://docs.databricks.com/aws/en/genie/knowledge-store)
